![baner_dogs](http://www.mf-data-science.fr/images/projects/dogs.jpg)
# <div style="padding:5px; margin-top:-40px; background: #343434; border-bottom: 2px solid #e4bf13; color:white; text-align:center !important;" id="intro">Détection de race de chien<br/>à partir d'images grâce aux <br/>réseaux de neurones à convolution</div>

L'objectif de ce Notebook est de détailler la mise en place d'un **algorithme de détection de la race du chien sur une photo**, afin d'accélérer le travail d’indexation dans une base de données.

### Les contraintes imposées :
- **Pré-processer les images** avec des techniques spécifiques *(e.g. whitening, equalization, éventuellement modification de la taille des images)*.
- Réaliser de la **data augmentation** *(mirroring, cropping...)*.
- Mise en oeuvre de 2 approches de l'utilisation des CNN :
    - Réaliser un réseau de neurones CNN from scratch en optimisant les paramètres.     
    - Utiliser le transfert learning et ainsi utiliser un réseau déjà entrainé.

# <span style="color:#343434" id="sommaire">Sommaire</span>
1. [Preprocessing des images](#section_1)     
    1.1. [Visualisation de la liste des races (classes) et un exemple de données](#section_1_1)     
    1.2. [Modification des histogrammes des images](#section_1_2)

In [ ]:
# Import libraries
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv

In [ ]:
warnings.filterwarnings('ignore')

# <span style="color:#343434" id="section_1">Preprocessing des images</span>

Pour commencer, nous allons rapidement analyser les données en regardant notamment l'état de répartition des races de chiens dans le répertoire images :

## <span style="color:#343434" id="section_1_1">Visualisation de la liste des races *(classes)* et un exemple de données.</span>

In [ ]:
# Define path to data
annotations_dir = '../input/stanford-dogs-dataset/annotations/Annotation' 
images_dir = '../input/stanford-dogs-dataset/images/Images'

In [ ]:
# Count the number of classes (dogs breeds)
breed_list = os.listdir(images_dir)
print("Number of breeds in dataset:", (len(breed_list)))

120 races de chien sont donc présentes dans notre jeu de données, ce qui représente **120 classes pour notre classifier**.
Nous allons à présent compter le nombre d'images de chaque race afin de vérifier si la distribution est équitable entre les classes :

In [ ]:
# Count number of pictures for each breed
df_breeds = pd.DataFrame(
    index=[breed.split('-',1)[1]
           for breed in breed_list],
    data=[len(os.listdir(images_dir + "/" + name))
          for name in breed_list],
    columns=["num_pictures"])

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(25,12))
df_breeds.plot(kind="bar",
               legend=False,
               ax=ax)
ax.axhline(df_breeds["num_pictures"].mean(),
           color='r', alpha=.7,
           linestyle='--',
           label="Mean of pictures")
plt.title("Number of pictures for each "\
          "dogs breeds of Dataset",
          color="#343434", fontsize=22)
plt.legend()
plt.show()

On remarque que les races de chien sont toutes bien alimentées en images. La moyenne se situe à 171 photos par classe. Aucune race n'est sous représentée nous pouvons donc toutes les conserver pour le moment.

**Regardons quelques exemples des photos par races** disponibles dans la base d'étude :

In [ ]:
def show_images_classes(path, classes, num_sample):
    """This function is used to display the first 
    n images of a directory passed as an argument. 
    It is adapted to subdirectories. 
    
    The matplotlib.image library must be loaded 
    with the alias mpimg. 

    Parameters
    ----------------------------------------
    path : string
        Link of root directory
    classes : string 
        Name of the subdirectory
    num_smaple : integer
        Number of picture to show
    ----------------------------------------
    """
    fig = plt.figure(figsize=(20,20))
    fig.patch.set_facecolor('#343434')
    plt.suptitle("{}".format(classes.split("-")[1]), y=.83,
                 color="white", fontsize=22)
    images = os.listdir(path + "/" + classes)[:num_sample]
    for i in range(num_sample):
        img = mpimg.imread(path+"/"+classes+"/"+images[i])
        plt.subplot(num_sample/5+1, 5, i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

In [ ]:
for i in np.random.randint(0, len(breed_list), size=3):
    show_images_classes(images_dir, breed_list[i], 5)

Avec un set d'image relativement important, les expositions, contraste, ... sont relativement différents pour chaque photo. Nous allons à présent utiliser des méthodes basées sur les histogrammes de ces images pour pre-processer au mieux ces données.

## <span style="color:#343434" id="section_1_2">Modification des histogrammes des images</span>

L'histogramme d'une image numérique est une courbe statistique représentant la **répartition de ses pixels selon leur intensité**. Commençons par regarder une image en particulier.

Nous allons transformer l'image dans différents codages couleurs. Le système de codage **YUV** est créé depuis une source RVB. Il est codé en trois composantes : **Y** représente la luminance *(informations de luminosité)* tandis que les deux autres (**U** et **V**) sont des données de chrominance *(informations de couleur)*. Ce format nous permet de visualiser au mieux l'histogramme pour les 3 dimensions :

In [ ]:
# Define test image
img_test = (images_dir 
            + "/" 
            + "n02085782-Japanese_spaniel/n02085782_1626.jpg")
img_test = cv.imread(img_test)

# Transform image with differents color sets
img_RGB = cv.cvtColor(img_test, cv.COLOR_BGR2RGB)
img_grayscale = cv.cvtColor(img_test, cv.COLOR_RGB2GRAY)
img_YUV = cv.cvtColor(img_test,cv.COLOR_BGR2YUV)

In [ ]:
# Create histogram
def plot_histogram(init_img, convert_img):
    hist, bins = np.histogram(
                    convert_img[1].flatten(),
                    256, [0,256])
    # Cumulative Distribution Function
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()

    # Plot histogram
    fig = plt.figure(figsize=(25,6))
    plt.subplot(1, 3, 1)
    plt.imshow(init_img[1])
    plt.title("{} Image".format(init_img[0]), 
              color="#343434")
    plt.subplot(1, 3, 2)
    plt.imshow(convert_img[1])
    plt.title("{} Image".format(convert_img[0]), 
              color="#343434")
    plt.subplot(1, 3, 3)
    plt.plot(cdf_normalized, 
             color='r', alpha=.7,
             linestyle='--')
    plt.hist(convert_img[1].flatten(),256,[0,256])
    plt.xlim([0,256])
    plt.legend(('cdf','histogram'), loc = 'upper left')
    plt.title("Histogram", color="#343434")
    plt.suptitle("Histogram and cumulative "\
                 "distribution for test image",
              color="black", fontsize=22, y=.98)
    plt.show()

In [ ]:
plot_histogram(["RGB", img_RGB], ["YUV", img_YUV])

On constate ici des pics importants au centre de l'histogram. Dans le cadre d'une bonne égalisation (amélioration du contraste), il est nécessaire de répartir la lumière dans tout le spectre de l'image. 

**Testons l'égalisation avec OpenCV :**     
L'intérêt de convertir l'image dans l'espace colorimétrique YUV est de pouvoir agir sur le canal "luminance" (Y) indépendamment des autres canaux de chrominance. Nous allons donc réaliser l'égalisation sur ce seul canal Y :

In [ ]:
# Equalization
img_YUV[:,:,0] = cv.equalizeHist(img_YUV[:,:,0])
img_equ = cv.cvtColor(img_YUV, cv.COLOR_YUV2RGB)
plot_histogram(["RGB", img_RGB], ["Equalized", img_equ])